In [32]:
pip install sentence-transformers


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 777.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [37]:
!pip install pdfplumber langdetect sentence-transformers transformers

In [38]:
# ============================
# 1. Gerekli Kütüphaneler
# ============================
# Colab için:
# !pip install pdfplumber langdetect sentence-transformers transformers

import pdfplumber
from langdetect import detect
from sentence_transformers import SentenceTransformer, util
from transformers import pipeline

# ============================
# 2. Anahtar Kelimeler ve Ağırlıklar
# ============================
criteria_labels = {
    "iş_deneyimi": ["İş Deneyimi", "Work Experience", "Career History", "Experience"],
    "egitim": ["Eğitim", "Education", "Academic Background", "University"],
    "teknik_beceriler": ["Teknik Beceriler", "Technical Skills", "Skills"],
    "ozet": ["Özet", "Hakkımda", "Summary", "Profile", "Objective"],
    "liderlik": ["Liderlik", "Leadership", "Organization Experience"],
    "sertifikalar": ["Sertifikalar", "Certificates", "Certifications", "Courses"],
    "iletisim": ["İletişim", "Contact", "Phone", "Email"],
    "portfolyo": ["Portfolyo", "Portfolio", "GitHub", "Website"],
    "diller": ["Diller", "Languages"],
    "referanslar": ["Referanslar", "References"]
}

criteria_weights = {
    "iş_deneyimi": 25,
    "egitim": 15,
    "teknik_beceriler": 15,
    "ozet": 10,
    "liderlik": 10,
    "sertifikalar": 10,
    "iletisim": 5,
    "portfolyo": 5,
    "diller": 3,
    "referanslar": 2
}

# ============================
# 3. Modellerin Yüklenmesi
# ============================
semantic_model = SentenceTransformer('all-MiniLM-L6-v2')  # Hızlı ve çok dilli model
classifier = pipeline("zero-shot-classification", model="MoritzLaurer/mDeBERTa-v3-base-mnli-xnli")

# ============================
# 4. PDF Metin Çıkarma
# ============================
def extract_text_from_pdf(file_path):
    text = ""
    with pdfplumber.open(file_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# ============================
# 5. Hibrit Analiz Fonksiyonu
# ============================
def analyze_cv(text):
    lang = detect(text)
    text_lower = text.lower()
    total_score = 0
    strengths, weaknesses = [], []

    for section, labels in criteria_labels.items():
        max_score = 0

        # --- 5.1 Keyword Kontrolü ---
        for kw in labels:
            if kw.lower() in text_lower:
                max_score = 1.0
                break

        # --- 5.2 Semantik Benzerlik ---
        if max_score < 1.0:
            embeddings1 = semantic_model.encode(text, convert_to_tensor=True)
            embeddings2 = semantic_model.encode(" ".join(labels), convert_to_tensor=True)
            sim = float(util.cos_sim(embeddings1, embeddings2))
            if sim > 0.4:
                max_score = max(max_score, sim)

        # --- 5.3 Zero-shot Kontrol ---
        if max_score < 0.4:  # yalnızca düşük benzerlik varsa çalıştır
            try:
                result = classifier(text[:512], candidate_labels=labels, multi_label=True)
                zs_score = max(result["scores"]) if "scores" in result else 0
                max_score = max(max_score, zs_score)
            except:
                pass  # zero-shot başarısız olursa devam et

        # --- 5.4 Puanlama ---
        section_score = round(criteria_weights[section] * max_score, 2)
        total_score += section_score

        if max_score >= 0.6:
            strengths.append(f"{section.replace('_', ' ').title()} bölümü iyi (benzerlik: {max_score:.2f}).")
        elif max_score >= 0.4:
            weaknesses.append(f"{section.replace('_', ' ').title()} bölümü zayıf (benzerlik: {max_score:.2f}).")
        else:
            weaknesses.append(f"{section.replace('_', ' ').title()} bölümü eksik.")

    total_score = round(total_score, 2)

    return {
        "language": lang,
        "total_score": total_score,
        "strengths": strengths,
        "weaknesses": weaknesses
    }

# ============================
# 6. Manuel PDF Yükleme
# ============================
def manual_pdf_upload():
    try:
        from google.colab import files
        print("Lütfen analiz etmek istediğiniz PDF dosyasını seçin:")
        uploaded = files.upload()
        pdf_path = list(uploaded.keys())[0]
        print(f"'{pdf_path}' başarıyla yüklendi.")
        return pdf_path
    except ImportError:
        pdf_path = input("PDF dosyasının tam yolunu girin: ")
        return pdf_path

# ============================
# 7. Ana Program
# ============================
if __name__ == "__main__":
    pdf_path = manual_pdf_upload()
    cv_text = extract_text_from_pdf(pdf_path)
    report = analyze_cv(cv_text)

    print(f"\nCV Dili: {report['language']}")
    print(f"Toplam Puan: {report['total_score']}/100\n")

    print("Güçlü Yönler:")
    for s in report['strengths']:
        print(f" - {s}")

    print("\nZayıf Yönler:")
    for w in report['weaknesses']:
        print(f" - {w}")


config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spm.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/286 [00:00<?, ?B/s]

Device set to use cpu


Lütfen analiz etmek istediğiniz PDF dosyasını seçin:


Saving Curriculum Vitae - Meriç Özcan.pdf to Curriculum Vitae - Meriç Özcan (8).pdf
'Curriculum Vitae - Meriç Özcan (8).pdf' başarıyla yüklendi.

CV Dili: en
Toplam Puan: 88.74/100

Güçlü Yönler:
 - Iş Deneyimi bölümü iyi (benzerlik: 1.00).
 - Egitim bölümü iyi (benzerlik: 1.00).
 - Teknik Beceriler bölümü iyi (benzerlik: 1.00).
 - Liderlik bölümü iyi (benzerlik: 1.00).
 - Iletisim bölümü iyi (benzerlik: 1.00).
 - Portfolyo bölümü iyi (benzerlik: 1.00).
 - Diller bölümü iyi (benzerlik: 1.00).
 - Referanslar bölümü iyi (benzerlik: 1.00).

Zayıf Yönler:
 - Ozet bölümü zayıf (benzerlik: 0.48).
 - Sertifikalar bölümü eksik.
